In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-


#################
## Import modules
#################
import sys
# walk directories
import glob
# access to OS functionality
import os
# call processes
import subprocess
# copy things
import copy
# numpy
import numpy as np
# opencv
#import cv2
# open3d
import open3d
#from lineset import LineMesh
# matplotlib for colormaps
import matplotlib.cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# scipy
from scipy import interpolate
# struct for reading binary ply files
import struct
# parse arguments
import argparse
try:
    import matplotlib.colors
    from PIL import __version__ as PILLOW_VERSION
    from PIL import Image
except:
    pass

#################
## Helper classes
#################

# annotation helper
from helpers.annotation  import Annotation3D, Annotation3DPly, global2local
from helpers.project     import Camera
from helpers.labels      import name2label, id2label, kittiId2label

ModuleNotFoundError: No module named 'open3d'

In [2]:
# the main class that parse fused point clouds
class Kitti360Viewer3D(object):

    # Constructor
    def __init__(self, seq=0, showStatic=True):

        # The sequence of the image we currently working on
        self.currentSequence   = ""
        # Image extension
        self.imageExt          = ".png"
        # Filenames of all images in current city
        self.images            = []
        self.imagesCityFull    = []
        # Ground truth type
        self.gtType            = 'semantic'
        # Add contour to semantic map
        self.semanticCt        = True
        # Add contour to instance map
        self.instanceCt        = True
        # The object that is highlighted and its label. An object instance
        self.highlightObj      = None
        self.highlightObjSparse= None
        self.highlightObjLabel = None
        # The current object the mouse points to. It's index in self.labels
        self.mouseObj          = -1
        # The current object the mouse points to. It's index in self.labels
        self.mousePressObj     = -1
        self.mouseSemanticId   = -1
        self.mouseInstanceId   = -1
        # show camera or not
        self.showCamera        = False
        self.downSampleEvery   = -1 
        # show bbox wireframe or mesh
        self.showWireframe     = False
        self.show3DInstanceOnly= True
        # show static or dynamic point clouds
        self.showStatic        = showStatic 
        # show visible point clouds only
        self.showVisibleOnly   = False
        # colormap
        self.cmap = matplotlib.cm.get_cmap('Set1')
        self.cmap_length = 9 


        kitti360Path = os.path.realpath("/Volumes/Erik's Files/Project/Datasets/KITTI-360/")
        

        sequence = '2013_05_28_drive_%04d_sync' % seq
        self.label3DPcdPath  = os.path.join(kitti360Path, 'data_3d_semantics')
        #self.annotation3D = Annotation3D(self.label3DBboxPath, sequence)
        self.annotation3DPly = Annotation3DPly(self.label3DPcdPath, sequence)
        self.sequence = sequence

        self.pointClouds = {}
        self.Rz = np.eye(3)
        self.accumuData = []

    def getColor(self, idx):
        if idx==0:
            return np.array([0,0,0])
        return np.asarray(self.cmap(idx % self.cmap_length)[:3])*255.

    def assignColor(self, globalIds, gtType='semantic'):
        if not isinstance(globalIds, (np.ndarray, np.generic)):
            globalIds = np.array(globalIds)[None]
        color = np.zeros((globalIds.size, 3))
        for uid in np.unique(globalIds):
            semanticId, instanceId = global2local(uid)
            if gtType=='semantic':
                color[globalIds==uid] = id2label[semanticId].color
            elif instanceId>0:
                color[globalIds==uid] = self.getColor(instanceId)
            else:
                color[globalIds==uid] = (96,96,96) # stuff objects in instance mode
        color = color.astype(np.float)/255.0
        return color
    
    def getLabelFilename(self, currentFile):
        # Generate the filename of the label file
        filename = os.path.basename(currentFile)
        search = [lb for lb in self.label_images if filename in lb]
        if not search:
            return ""
        filename = os.path.normpath(search[0])
        return filename


    def loadWindow(self, pcdFile, colorType='semantic', isLabeled=True, isDynamic=False):
        window = pcdFile.split('/')[-2]
        
        print ('Loading %s ' % pcdFile)
        
        # load ply data using open3d for visualization
        if window in self.pointClouds.keys():
            pcd = self.pointClouds[window]
        else:
            pcd = open3d.io.read_point_cloud(pcdFile)

        n_pts = np.asarray(pcd.points).shape[0]
        data = self.annotation3DPly.readBinaryPly(pcdFile, n_pts)
        
        color=data[:,3:6]
        mask = np.logical_and(np.mean(color,1)==128, np.std(color,1)==0)
        mask = 1-mask
        pcd = pcd.select_by_index(np.where(mask)[0])

        data = data[mask.astype(np.bool),:]
        self.accumuData.append(data)


        # assign color
        if colorType=='semantic' or colorType=='instance':
            globalIds = data[:,7]
        
            ptsColor = self.assignColor(globalIds, colorType)
            pcd.colors = open3d.utility.Vector3dVector(ptsColor)
        elif colorType!='rgb':
            raise ValueError("Color type can only be 'rgb','semantic', 'instance'!")

        return pcd


    def loadWindows(self, colorType='semantic'):
        pcdFolder = 'static' if self.showStatic else 'dynamic'
        pcdFileList = sorted(glob.glob(os.path.join(self.label3DPcdPath, self.sequence, pcdFolder, '*.ply')))
    
        if not len(pcdFileList):
            print ('%s does not exist!!' % os.path.join(self.label3DPcdPath, self.sequence, '*', pcdName))
            return None

        for idx,pcdFile in enumerate(pcdFileList):

            window = pcdFile.split('/')[-2]
            pcd = self.loadWindow(pcdFile)

            self.pointClouds[window] = pcd

## Sequence Visualizer

In [ ]:
class opts():
    sequence = 0
    mode = 'semantic'
    max_bbox = 100

args = opts()

v = Kitti360Viewer3D(args.sequence, False)
pcdFile = v.annotation3DPly.pcdFileList[30]
pcd = v.loadWindow(pcdFile, args.mode)

# Visualize the last sample scene
mesh_frame = open3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=-pcd.get_center())
open3d.visualization.draw_geometries([pcd, mesh_frame])

Found 44 ply files in 2013_05_28_drive_0000_sync
Loading /Volumes/Erik's Files/Project/Datasets/KITTI-360/data_3d_semantics/2013_05_28_drive_0000_sync/static/007777_007982.ply 


## Mini Dataset Creator

In [4]:
from collections import namedtuple
from statistics import mode
import math


#--------------------------------------------------------------------------------
# Definitions
#--------------------------------------------------------------------------------

# a label and all meta information
Label = namedtuple( 'Label' , [

    'name'        , # The identifier of this label, e.g. 'car', 'person', ... .
                    # We use them to uniquely name a class

    'id'          , # An integer ID that is associated with this label.
                    # The IDs are used to represent the label in ground truth images
                    # An ID of -1 means that this label does not have an ID and thus
                    # is ignored when creating ground truth images (e.g. license plate).
                    # Do not modify these IDs, since exactly these IDs are expected by the
                    # evaluation server.

    'kittiId'     , # An integer ID that is associated with this label for KITTI-360
                    # NOT FOR RELEASING

    'trainId'     , # Feel free to modify these IDs as suitable for your method. Then create
                    # ground truth images with train IDs, using the tools provided in the
                    # 'preparation' folder. However, make sure to validate or submit results
                    # to our evaluation server using the regular IDs above!
                    # For trainIds, multiple labels might have the same ID. Then, these labels
                    # are mapped to the same class in the ground truth images. For the inverse
                    # mapping, we use the label that is defined first in the list below.
                    # For example, mapping all void-type classes to the same ID in training,
                    # might make sense for some approaches.
                    # Max value is 255!

    'category'    , # The name of the category that this label belongs to

    'categoryId'  , # The ID of this category. Used to create ground truth images
                    # on category level.

    'hasInstances', # Whether this label distinguishes between single instances or not

    'ignoreInEval', # Whether pixels having this class as ground truth label are ignored
                    # during evaluations or not

    'color'       , # The color of this label
    ] )


#--------------------------------------------------------------------------------
# A list of all labels
#--------------------------------------------------------------------------------

# Please adapt the train IDs as appropriate for your approach.
# Note that you might want to ignore labels with ID 255 during training.
# Further note that the current train IDs are only a suggestion. You can use whatever you like.
# Make sure to provide your results using the original IDs and not the training IDs.
# Note that many IDs are ignored in evaluation and thus you never need to predict these!

labels = [
    #       name                     id    kittiId,    trainId   category            catId     hasInstances   ignoreInEval   color
    Label(  'unlabeled'            ,  0 ,       -1 ,       255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'ego vehicle'          ,  1 ,       -1 ,       255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'rectification border' ,  2 ,       -1 ,       255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'out of roi'           ,  3 ,       -1 ,       255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'static'               ,  4 ,       -1 ,       255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'dynamic'              ,  5 ,       -1 ,       255 , 'void'            , 0       , False        , True         , (111, 74,  0) ),
    Label(  'ground'               ,  6 ,       -1 ,       255 , 'void'            , 0       , False        , True         , ( 81,  0, 81) ),
    Label(  'road'                 ,  7 ,        1 ,         0 , 'flat'            , 1       , False        , False        , (128, 64,128) ),
    Label(  'sidewalk'             ,  8 ,        3 ,         1 , 'flat'            , 1       , False        , False        , (244, 35,232) ),
    Label(  'parking'              ,  9 ,        2 ,       255 , 'flat'            , 1       , False        , True         , (250,170,160) ),
    Label(  'rail track'           , 10 ,        10,       255 , 'flat'            , 1       , False        , True         , (230,150,140) ),
    Label(  'building'             , 11 ,        11,         2 , 'construction'    , 2       , True         , False        , ( 70, 70, 70) ),
    Label(  'wall'                 , 12 ,        7 ,         3 , 'construction'    , 2       , False        , False        , (102,102,156) ),
    Label(  'fence'                , 13 ,        8 ,         4 , 'construction'    , 2       , False        , False        , (190,153,153) ),
    Label(  'guard rail'           , 14 ,        30,       255 , 'construction'    , 2       , False        , True         , (180,165,180) ),
    Label(  'bridge'               , 15 ,        31,       255 , 'construction'    , 2       , False        , True         , (150,100,100) ),
    Label(  'tunnel'               , 16 ,        32,       255 , 'construction'    , 2       , False        , True         , (150,120, 90) ),
    Label(  'pole'                 , 17 ,        21,         5 , 'object'          , 3       , True         , False        , (153,153,153) ),
    Label(  'polegroup'            , 18 ,       -1 ,       255 , 'object'          , 3       , False        , True         , (153,153,153) ),
    Label(  'traffic light'        , 19 ,        23,         6 , 'object'          , 3       , True         , False        , (250,170, 30) ),
    Label(  'traffic sign'         , 20 ,        24,         7 , 'object'          , 3       , True         , False        , (220,220,  0) ),
    Label(  'vegetation'           , 21 ,        5 ,         8 , 'nature'          , 4       , False        , False        , (107,142, 35) ),
    Label(  'terrain'              , 22 ,        4 ,         9 , 'nature'          , 4       , False        , False        , (152,251,152) ),
    Label(  'sky'                  , 23 ,        9 ,        10 , 'sky'             , 5       , False        , False        , ( 70,130,180) ),
    Label(  'person'               , 24 ,        19,        11 , 'human'           , 6       , True         , False        , (220, 20, 60) ),
    Label(  'rider'                , 25 ,        20,        12 , 'human'           , 6       , True         , False        , (255,  0,  0) ),
    Label(  'car'                  , 26 ,        13,        13 , 'vehicle'         , 7       , True         , False        , (  0,  0,142) ),
    Label(  'truck'                , 27 ,        14,        14 , 'vehicle'         , 7       , True         , False        , (  0,  0, 70) ),
    Label(  'bus'                  , 28 ,        34,        15 , 'vehicle'         , 7       , True         , False        , (  0, 60,100) ),
    Label(  'caravan'              , 29 ,        16,       255 , 'vehicle'         , 7       , True         , True         , (  0,  0, 90) ),
    Label(  'trailer'              , 30 ,        15,       255 , 'vehicle'         , 7       , True         , True         , (  0,  0,110) ),
    Label(  'train'                , 31 ,        33,        16 , 'vehicle'         , 7       , True         , False        , (  0, 80,100) ),
    Label(  'motorcycle'           , 32 ,        17,        17 , 'vehicle'         , 7       , True         , False        , (  0,  0,230) ),
    Label(  'bicycle'              , 33 ,        18,        18 , 'vehicle'         , 7       , True         , False        , (119, 11, 32) ),
    Label(  'garage'               , 34 ,        12,         2 , 'construction'    , 2       , True         , False        , ( 64,128,128) ),
    Label(  'gate'                 , 35 ,        6 ,         4 , 'construction'    , 2       , False        , False        , (190,153,153) ),
    Label(  'stop'                 , 36 ,        29,       255 , 'construction'    , 2       , True         , True         , (150,120, 90) ),
    Label(  'smallpole'            , 37 ,        22,         5 , 'object'          , 3       , True         , False        , (153,153,153) ),
    Label(  'lamp'                 , 38 ,        25,       255 , 'object'          , 3       , True         , False        , (0,   64, 64) ),
    Label(  'trash bin'            , 39 ,        26,       255 , 'object'          , 3       , True         , False        , (0,  128,192) ),
    Label(  'vending machine'      , 40 ,        27,       255 , 'object'          , 3       , True         , False        , (128, 64,  0) ),
    Label(  'box'                  , 41 ,        28,       255 , 'object'          , 3       , True         , False        , (64,  64,128) ),
    Label(  'unknown construction' , 42 ,        35,       255 , 'void'            , 0       , False        , True         , (102,  0,  0) ),
    Label(  'unknown vehicle'      , 43 ,        36,       255 , 'void'            , 0       , False        , True         , ( 51,  0, 51) ),
    Label(  'unknown object'       , 44 ,        37,       255 , 'void'            , 0       , False        , True         , ( 32, 32, 32) ),
    Label(  'license plate'        , -1 ,        -1,        -1 , 'vehicle'         , 7       , False        , True         , (  0,  0,142) ),
]

#--------------------------------------------------------------------------------
# Create dictionaries for a fast lookup
#--------------------------------------------------------------------------------

# color to classification
color2labelid = { label.color:label.kittiId for label in labels}

# classification to color
labelid2color = { label.kittiId:label.color for label in labels}

#--------------------------------------------------------------------------------
# Create function to convert segmentation colors to classification
#--------------------------------------------------------------------------------
def downsapler(part, part_sem, voxel_size=0.1):
    # input is numpy array N * xyzrgblll where lll are the segmentation colors for each class
    # output are 3 numpy arrays: N * xyz, N * rgb, N * labels
    part_vxds = part.voxel_down_sample_and_trace(voxel_size, part.get_min_bound(), part.get_max_bound())
    part_down_sampled = part_vxds[0]

    part_sem_vxds = part_sem.voxel_down_sample_and_trace(voxel_size, part_sem.get_min_bound(), part_sem.get_max_bound())
    part_sem_down_sampled = part_sem_vxds[0]
    part_sem_traces = part_sem_vxds[2]

    labels = []

    for idx in part_sem_traces:
        idx = list(idx)
        colors = np.asarray(part_sem.select_by_index(idx).colors)
        mode_color_array = (255*colors).astype(int)
        mode_color_tuples = [tuple(x) for x in mode_color_array]

        # you cant use scipy.stats.mode becasue when there isnt a mode, it picks the mins for each column!
        try:
            voxel_colors_mode = mode(mode_color_tuples)
        except:
            voxel_colors_mode = mode_color_tuples[0]
        
        labelid = color2labelid[tuple(voxel_colors_mode)]
        labels.append(int(labelid))

    points = np.concatenate((np.asarray(part_down_sampled.points), np.asarray(part_down_sampled.colors)), axis=1)
    return points, np.array(labels)

In [14]:
# Create mini dataset with only the street and sidewalks for training and testing the model
scene_step = 2

class opts():
    sequence = 0
    mode = 'rgb'

args = opts()


v = Kitti360Viewer3D(args.sequence, True)
pcdFile = v.annotation3DPly.pcdFileList[20]

# pcd.colors are the rgb of data
pcd = v.loadWindow(pcdFile, 'rgb')

# pcd.colors are the semantic segmentation data
pcd_sem = v.loadWindow(pcdFile, 'semantic')


d = 14
t = math.radians(d)
x = math.cos(t)
y = math.sin(t)
pcd.rotate(np.array([[x,-y,0],[y,x,0],[0,0,1]]), pcd.get_center())

# to get the labels back after rotations!
pcd_sem.rotate(np.array([[x,-y,0],[y,x,0],[0,0,1]]), pcd_sem.get_center())

box = open3d.geometry.AxisAlignedBoundingBox.create_from_points(pcd.points)

for i in range(35):
    vol = open3d.geometry.OrientedBoundingBox.create_from_axis_aligned_bounding_box(box)
    vol.extent = np.array([27,20,10])
    vol.center = vol.get_center() - np.array([22, (30 - i*scene_step), 0])
    part = pcd.crop(vol)
    part_sem = pcd_sem.crop(vol)
    
    # convert semantic colors to labels
    points, labels = downsapler(part, part_sem, 0.2)
    file_name = 'mini_dataset_big_vx02/mini_dataset_xyzrgbl_' + str(i)
    
    data = np.concatenate((points, labels.reshape([labels.shape[0],1])), axis=1)
    np.save(file_name, data)
    #print(points.shape)

    #Visualize the last sample scene
#     mesh_frame = open3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=-pcd.get_center())
#     px = open3d.geometry.PointCloud(open3d.utility.Vector3dVector(points[:,0:3]))
#     open3d.visualization.draw_geometries([px, mesh_frame])
#     break

Found 44 ply files in 2013_05_28_drive_0000_sync
Loading /Volumes/Erik's Files/Project/Datasets/KITTI-360/data_3d_semantics/2013_05_28_drive_0000_sync/static/005249_005900.ply 
Loading /Volumes/Erik's Files/Project/Datasets/KITTI-360/data_3d_semantics/2013_05_28_drive_0000_sync/static/005249_005900.ply 


In [6]:
mesh_frame = open3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=-pcd.get_center())
px = open3d.geometry.PointCloud(open3d.utility.Vector3dVector(points[:,0:3]))
open3d.visualization.draw_geometries([px, mesh_frame])

In [25]:
open3d.visualization.draw_geometries([part_sem, mesh_frame])

## Mini Dataset Creator: Full Resolution, for random sampling

In [6]:
# Create mini dataset with only the street and sidewalks for training and testing the model
scene_step = 2

class opts():
    sequence = 0
    mode = 'rgb'

args = opts()


v = Kitti360Viewer3D(args.sequence, True)
pcdFile = v.annotation3DPly.pcdFileList[20]

# pcd.colors are the rgb of data
pcd = v.loadWindow(pcdFile, 'rgb')

# pcd.colors are the semantic segmentation data
pcd_sem = v.loadWindow(pcdFile, 'semantic')


d = 14
t = math.radians(d)
x = math.cos(t)
y = math.sin(t)
pcd.rotate(np.array([[x,-y,0],[y,x,0],[0,0,1]]), pcd.get_center())

# to get the labels back after rotations!
pcd_sem.rotate(np.array([[x,-y,0],[y,x,0],[0,0,1]]), pcd_sem.get_center())

box = open3d.geometry.AxisAlignedBoundingBox.create_from_points(pcd.points)

for i in range(35):
    vol = open3d.geometry.OrientedBoundingBox.create_from_axis_aligned_bounding_box(box)
    vol.extent = np.array([20,20,10])
    vol.center = vol.get_center() - np.array([23.5, (30 - i*scene_step), 0])
    part = pcd.crop(vol)
    part_sem = pcd_sem.crop(vol)
    
    # convert semantic colors to labels
    points = np.asarray(part.points)
    rgb = np.asarray(part.colors)
    label_colors = np.asarray(part_sem.colors)*255
    labels = np.array([color2labelid[tuple(x.astype(int))] for x in label_colors])
    file_name = 'mini_dataset_fullres/mini_dataset_fullres_xyzrgbl_' + str(i)
    
    data = np.concatenate((points, rgb, labels.reshape([labels.shape[0],1])), axis=1)
    np.save(file_name, data)
    #print(labels)
    #Visualize the last sample scene
    #mesh_frame = open3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=-pcd.get_center())
    #px = open3d.geometry.PointCloud(open3d.utility.Vector3dVector(points[:,0:3]))
    #open3d.visualization.draw_geometries([px, mesh_frame])
    #break

Found 44 ply files in 2013_05_28_drive_0000_sync
Loading /Volumes/Erik's Files/Project/Datasets/KITTI-360/data_3d_semantics/2013_05_28_drive_0000_sync/static/005249_005900.ply 
Loading /Volumes/Erik's Files/Project/Datasets/KITTI-360/data_3d_semantics/2013_05_28_drive_0000_sync/static/005249_005900.ply 


In [5]:
# Create mini dataset with only the street and sidewalks for training and testing the model
scene_step = 2

class opts():
    sequence = 0
    mode = 'rgb'

args = opts()


v = Kitti360Viewer3D(args.sequence, True)
pcdFile = v.annotation3DPly.pcdFileList[30]

# pcd.colors are the rgb of data
pcd = v.loadWindow(pcdFile, 'rgb')

# pcd.colors are the semantic segmentation data
pcd_sem = v.loadWindow(pcdFile, 'semantic')


d = -64
t = math.radians(d)
x = math.cos(t)
y = math.sin(t)
pcd.rotate(np.array([[x,-y,0],[y,x,0],[0,0,1]]), pcd.get_center())

# to get the labels back after rotations!
pcd_sem.rotate(np.array([[x,-y,0],[y,x,0],[0,0,1]]), pcd_sem.get_center())

box = open3d.geometry.AxisAlignedBoundingBox.create_from_points(pcd.points)

for i in range(50):
    vol = open3d.geometry.OrientedBoundingBox.create_from_axis_aligned_bounding_box(box)
    vol.extent = np.array([27,20,10])
    vol.center = vol.get_center() - np.array([6.5, (30 - i*scene_step), 1])
    part = pcd.crop(vol)
    part_sem = pcd_sem.crop(vol)
    
    
    # convert semantic colors to labels
    points = np.asarray(part.points)
    rgb = np.asarray(part.colors)
    label_colors = np.asarray(part_sem.colors)*255
    labels = np.array([color2labelid[tuple(x.astype(int))] for x in label_colors])
    file_name = 'mini_dataset_fullres/mini_dataset_fullres_2_xyzrgbl_' + str(i)
    
    data = np.concatenate((points, rgb, labels.reshape([labels.shape[0],1])), axis=1)
    np.save(file_name, data)
    #print(points.shape)
    
    # Visualize the last sample scene
#     mesh_frame = open3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=-pcd.get_center())
#     open3d.visualization.draw_geometries([part, mesh_frame])
#     break

Found 44 ply files in 2013_05_28_drive_0000_sync
Loading /Volumes/Erik's Files/Project/Datasets/KITTI-360/data_3d_semantics/2013_05_28_drive_0000_sync/static/007777_007982.ply 
Loading /Volumes/Erik's Files/Project/Datasets/KITTI-360/data_3d_semantics/2013_05_28_drive_0000_sync/static/007777_007982.ply 


## Mini Test Dataset Creator

In [22]:
# Create mini dataset with only the street and sidewalks for training and testing the model
scene_step = 2

class opts():
    sequence = 0
    mode = 'rgb'

args = opts()


v = Kitti360Viewer3D(args.sequence, True)
pcdFile = v.annotation3DPly.pcdFileList[30]

# pcd.colors are the rgb of data
pcd = v.loadWindow(pcdFile, 'rgb')

# pcd.colors are the semantic segmentation data
pcd_sem = v.loadWindow(pcdFile, 'semantic')


d = -64
t = math.radians(d)
x = math.cos(t)
y = math.sin(t)
pcd.rotate(np.array([[x,-y,0],[y,x,0],[0,0,1]]), pcd.get_center())

# to get the labels back after rotations!
pcd_sem.rotate(np.array([[x,-y,0],[y,x,0],[0,0,1]]), pcd_sem.get_center())

box = open3d.geometry.AxisAlignedBoundingBox.create_from_points(pcd.points)

for i in range(50):
    vol = open3d.geometry.OrientedBoundingBox.create_from_axis_aligned_bounding_box(box)
    vol.extent = np.array([27,20,10])
    vol.center = vol.get_center() - np.array([6.5, (30 - i*scene_step), 1])
    part = pcd.crop(vol)
    part_sem = pcd_sem.crop(vol)
    
    
    # convert semantic colors to labels
    points, labels = downsapler(part, part_sem, 0.2)
    file_name = 'mini_dataset_big_vx02/mini_dataset_2_xyzrgbl_' + str(i)
    
    data = np.concatenate((points, labels.reshape([labels.shape[0],1])), axis=1)
    np.save(file_name, data)
    #print(points.shape)
    
    # Visualize the last sample scene
#     mesh_frame = open3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=-pcd.get_center())
#     open3d.visualization.draw_geometries([part, mesh_frame])
#     break

Found 44 ply files in 2013_05_28_drive_0000_sync
Loading /Volumes/Erik's Files/Project/Datasets/KITTI-360/data_3d_semantics/2013_05_28_drive_0000_sync/static/007777_007982.ply 
Loading /Volumes/Erik's Files/Project/Datasets/KITTI-360/data_3d_semantics/2013_05_28_drive_0000_sync/static/007777_007982.ply 


In [6]:
mesh_frame = open3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=-pcd.get_center())
open3d.visualization.draw_geometries([part, mesh_frame])

## Random Code Tests

In [4]:
t0 = np.load('mini_dataset/mini_dataset_xyzrgbl_1.npy')

In [5]:
t0.shape

(40555, 7)

In [6]:
labelid2color = {label.id:label.color for label in labels}
v0 = open3d.utility.Vector3dVector(t0[:,0:3])
c0 = [np.array(labelid2color[x[-1]])/255 for x in t0]
v00 = open3d.utility.Vector3dVector(c0)
pcd2 = open3d.geometry.PointCloud(v0)
pcd2.colors = v00

In [39]:
open3d.visualization.draw_geometries([pcd2])

In [7]:
pcd2.estimate_normals(search_param=open3d.geometry.KDTreeSearchParamHybrid(radius=0.3, max_nn=6))
pcd2.orient_normals_to_align_with_direction()

True

In [8]:
open3d.visualization.draw_geometries([pcd2],point_show_normal=True)

In [353]:
ls = t0[:,6:7]*np.ones_like(t0[:,0:3])
v000 = open3d.utility.Vector3dVector(ls)
pcd2.colors = v000
open3d.visualization.draw_geometries([pcd2])

In [ ]:
t0[100:120,6:7]

In [439]:
v0 = open3d.utility.Vector3dVector(t0[:,0:3])
v00 = open3d.utility.Vector3dVector(t0[:,3:6])
pcd2 = open3d.geometry.PointCloud(v0)
pcd2.colors = v00
pcd2.voxel_down_sample(0.2)

geometry::PointCloud with 6045 points.

In [1137]:
p = part.voxel_down_sample_and_trace(0.15, part.get_min_bound(), part_sem.get_max_bound())
p_sem = part_sem.voxel_down_sample_and_trace(0.15, part_sem.get_min_bound(), part_sem.get_max_bound())
#print(p)
#print(p_sem)

### k-nearest

In [23]:
pcd_tree = open3d.geometry.KDTreeFlann(pcd2)
[k, idx, d] = pcd_tree.search_knn_vector_3d(pcd2.points[10], 4)
print(k,idx,d)

4 IntVector[10, 11, 22380, 22073] DoubleVector[0, 0.00615776, 0.0256391, 0.0573854]


In [28]:
K = []
IDX = []
D = []
for i in range(10000):
    [k, idx, d] = pcd_tree.search_knn_vector_3d(pcd2.points[i], 4)
    K.append(k)
    IDX.append(idx)
    D.append(d)

# VAE

In [6]:
Y = np.load('Y-2.npy')
# l = range(900,1000)
# pts = Y[0,l,0:3]*15
# rgb = Y[0,l,3:6]
pts = Y[0,:,0:3]#*15
rgb = Y[0,:,3:6]

In [7]:
xyz = open3d.utility.Vector3dVector(pts)
pcd_colors = open3d.utility.Vector3dVector(rgb)
pcd0 = open3d.geometry.PointCloud(xyz)
pcd0.colors = pcd_colors
open3d.visualization.draw_geometries([pcd0])

In [68]:
print(Y[0,1:50,:3])

[[ 2.3287973   0.5621117  -1.0827452 ]
 [-1.4845214  -0.03107503  2.6949065 ]
 [-1.0202713  -0.8881884  -2.2054584 ]
 [-0.25012538 -0.911252   -0.37897602]
 [-0.47461894 -0.03860363 -3.267272  ]
 [ 0.0913282  -0.43400007 -0.86597   ]
 [ 0.6835573  -1.1790788   1.0574652 ]
 [ 1.9914403   0.7279136  -0.9862106 ]
 [-0.08092725  0.25658014 -3.132513  ]
 [-0.9002515   1.3548651   1.4159157 ]
 [ 0.5760046  -1.0684519   1.4368594 ]
 [-0.77308905 -0.51228595  0.99722457]
 [ 0.7168331  -1.2190093   0.38112143]
 [-0.848552    1.1755533  -0.67953706]
 [ 1.5653814   0.85880697  0.23238173]
 [-0.5500667   0.93335474 -0.9400485 ]
 [-0.6444188   1.7142674   0.21598628]
 [ 0.02139163  0.49864778  1.2496477 ]
 [ 0.73099583  2.3985696  -1.2343806 ]
 [-1.1575047  -0.2927308   1.0882311 ]
 [-0.27650353 -0.39722785 -0.7650398 ]
 [-0.95080686 -0.51901895  1.270132  ]
 [-0.6045629   1.3012292   0.24706548]
 [ 0.56922424 -1.0770212   0.10777709]
 [ 0.28325832 -0.37196413 -0.97051644]
 [ 0.26134294  1.3118886 

## Prediction

In [2]:
pred = np.load('pred.npy')
points0 = np.load('points.npy')
batch_target = np.load('target.npy')
batch_points = points0
batch_pred = pred.reshape([8,16384,1])

#points_og = np.load('test_dataset/test_data.npy')

NameError: name 'np' is not defined

In [6]:
batch_points[:,:,0] = batch_points[:,:,0]*15 + 1187.0139032871741
batch_points[:,:,1] = batch_points[:,:,1]*15 + 3554.7363710009868
batch_points[:,:,2] = batch_points[:,:,2]*15 + 117.21941488949514

In [7]:
# Choose sample in batch
n = 0
pts = np.array(batch_points[n,:,0:3])
preds = batch_pred[n].squeeze(1)
#tgts = batch_target[n].squeeze(1)
colr = np.array(batch_points[n,:,3:6])

In [10]:
# Process sample predictions
xyz = open3d.utility.Vector3dVector(pts)
pred_rgb = [np.array(labelid2color[(x-1)])/255 for x in preds]
pcd_colors = open3d.utility.Vector3dVector(pred_rgb)
pcd0 = open3d.geometry.PointCloud(xyz)
pcd0.colors = pcd_colors

# Process sample targets
# tgts_rgb = [np.array(labelid2color[(x-1)])/255 for x in tgts]
# pcd_tgts_colors = open3d.utility.Vector3dVector(tgts_rgb)
# pcd0_tgts = open3d.geometry.PointCloud(xyz)
# pcd0_tgts.colors = pcd_tgts_colors

# # Process sample prediction vs targets difference
# diff = tgts-preds
# diff[diff != 0] = 1
# diff_dict = {0:(0.7,0.7,0.7), 1:(1,0,0)}
# diff_rgb = [diff_dict[x] for x in diff]
# pcd_diff_colors = open3d.utility.Vector3dVector(diff_rgb)
# pcd0_diff = open3d.geometry.PointCloud(xyz)
# pcd0_diff.colors = pcd_diff_colors

# Process model input
# xyz_og = open3d.utility.Vector3dVector(points_og[:,0:3])
# #og_rgb = [np.array(labelid2color[(x-1)])/255 for x in colors_og]
# pcd_og_colors = open3d.utility.Vector3dVector(points_og[:,3:6])
# pcd0_og = open3d.geometry.PointCloud(xyz_og)
# pcd0_og.colors = pcd_og_colors

# Process sample sampled input
pcd_samp_colors = open3d.utility.Vector3dVector(colr)
pcd0_samp = open3d.geometry.PointCloud(xyz)
pcd0_samp.colors = pcd_samp_colors

open3d.visualization.draw_geometries([pcd0])
#open3d.visualization.draw_geometries([pcd0_diff])
#open3d.visualization.draw_geometries([pcd0_tgts])
#open3d.visualization.draw_geometries([pcd0_samp])
#open3d.visualization.draw_geometries([pcd0_og, pcd0])

In [125]:
pcd0.estimate_normals(search_param=open3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=10))
pcd0.orient_normals_to_align_with_direction()
open3d.visualization.draw_geometries([pcd0],point_show_normal=True)

In [129]:
xyz = open3d.utility.Vector3dVector(pts)
pred_rgb = [np.array(labelid2color[(x-1)])/255 for x in preds]
pcd_colors = open3d.utility.Vector3dVector(pred_rgb)
pcd0 = open3d.geometry.PointCloud(xyz)
pcd0.colors = pcd_colors

In [35]:
sum(diff)

1161

In [14]:
sum(diff)

2102

## Random code test

In [ ]:
def query_ball_point(radius, nsample, xyz, new_xyz):
    """
    Input:
        radius: local region radius
        nsample: max sample number in local region
        xyz: all points, [B, N, 3]
        new_xyz: query points, [B, S, 3]
    Return:
        group_idx: grouped points index, [B, S, nsample]
    """
    device = xyz.device
    B, N, C = xyz.shape
    
    _, S, _ = new_xyz.shape
    group_idx = torch.arange(N, dtype=torch.long).to(device).view(1, 1, N).repeat([B, S, 1])
    sqrdists = square_distance(new_xyz, xyz)
    group_idx[sqrdists > radius ** 2] = N
    group_idx = group_idx.sort(dim=-1)[0][:, :, :nsample]
    group_first = group_idx[:, :, 0].view(B, S, 1).repeat([1, 1, nsample])
    mask = group_idx == N
    group_idx[mask] = group_first[mask]
    
    
    return group_idx

### Test for overlapping voxels

In [4]:
t0 = np.load('mini_dataset_vx_0_18/mini_dataset_xyzrgbl_30.npy')
#labelid2color = {label.id:label.color for label in labels}
v0 = open3d.utility.Vector3dVector(t0[:,0:3])
#c0 = [np.array(labelid2color[x[-1]])/255 for x in t0]
#v00 = open3d.utility.Vector3dVector(c0)
pcd = open3d.geometry.PointCloud(v0)
#pcd.colors = v00
open3d.visualization.draw_geometries([pcd])

In [284]:
pcd_tree = open3d.geometry.KDTreeFlann(pcd)

In [300]:
[k, idx, _] = pcd_tree.search_radius_vector_3d(pcd.points[1500], 0.2)

### Computing voxel_size for centroids

In [499]:
class opts():
    sequence = 0
    mode = 'semantic'
    max_bbox = 100

args = opts()

v = Kitti360Viewer3D(args.sequence, False)
pcdFile = v.annotation3DPly.pcdFileList[20]
pcd = v.loadWindow(pcdFile, args.mode)

d = 14
t = math.radians(d)
x = math.cos(t)
y = math.sin(t)
pcd.rotate(np.array([[x,-y,0],[y,x,0],[0,0,1]]), pcd.get_center())

box = open3d.geometry.AxisAlignedBoundingBox.create_from_points(pcd.points)

vol = open3d.geometry.OrientedBoundingBox.create_from_axis_aligned_bounding_box(box)
vol.extent = np.array([20,20,10])
vol.center = vol.get_center() - np.array([23.5, 30, 0])
part = pcd.crop(vol)

print(part)

Found 44 ply files in 2013_05_28_drive_0000_sync
Loading /Volumes/Erik's Files/Project/Datasets/KITTI-360/data_3d_semantics/2013_05_28_drive_0000_sync/static/005249_005900.ply 
geometry::PointCloud with 236010 points.


In [500]:
open3d.visualization.draw_geometries([part, mesh_frame])

In [412]:
_, _, trace = part.voxel_down_sample_and_trace(.7, part.get_min_bound(), part_sem.get_max_bound())

# Visualize the last sample scene
#mesh_frame = open3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=-pcd.get_center())
#open3d.visualization.draw_geometries([pcd, mesh_frame])

In [482]:
px = part.voxel_down_sample_and_trace(.7, part.get_min_bound(), part_sem.get_max_bound())

# Visualize the last sample scene
mesh_frame = open3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=-pcd.get_center())
open3d.visualization.draw_geometries([px[0], mesh_frame])

In [417]:
np.random.randint(0,len(trace), 1024)

array([496,  81, 715, ...,  91, 382, 860])

In [464]:
rand_idx = np.random.randint(0,len(trace), 1024)

In [465]:
tl=[trace[i] for i in rand_idx]

In [544]:
#pcd = op3.geometry.PointCloud( op3.utility.Vector3dVector( points_xyz[i] ) )
pcd_tree = open3d.geometry.KDTreeFlann(pcd)

[k, idx, z] = pcd_tree.search_radius_vector_3d([1310, 3779,  114] , 2)
idx = np.asarray(idx)

In [548]:
np.asarray(z).shape

(327,)

In [531]:
t[2]

116.33988265617411

In [552]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
import torch.nn.functional as F
from torch.utils.data import Dataset

import numpy as np
import matplotlib.pyplot as plt
import os, sys, importlib, shutil
import time, random, logging, datetime
from pathlib import Path
from tqdm import notebook
import glob
import open3d as op3
from IPython.display import clear_output

In [553]:
class KITTI360Dataset(Dataset):
    def __init__(self, data_root='test/', num_points=4096,
                 voxel_size=0.1, num_classes=46, transform=None):
        super().__init__()
        self.num_points = num_points
        self.voxel_size = voxel_size
        self.num_classes = num_classes
        self.transform = transform
        frames = sorted(os.listdir(data_root))
        frames = [frame for frame in frames]
                
        self.frame_points, self.frame_labels = [], []
        self.frame_coord_min, self.frame_coord_max = [], []
        num_point_all = []
        
        for frame_name in notebook.tqdm(frames, total=len(frames)):#, position=0, leave=True):
            frame_path = os.path.join(data_root, frame_name)
            frame_data = np.load(frame_path) # xyzrgblll, N*9

            # read data N * xyzrgbl
            points, labels = frame_data[:, 0:6], frame_data[:, 6:7]  # xyzrgb, N*6; l, N
            labels = labels + np.ones_like(labels)
            coord_min, coord_max = np.amin(points, axis=0)[:3], np.amax(points, axis=0)[:3]
            self.frame_points.append(points), self.frame_labels.append(labels)
            self.frame_coord_min.append(coord_min), self.frame_coord_max.append(coord_max)
            num_point_all.append(labels.shape[0])

        self.frame_idxs = np.arange(len(frames))
        print("Total of {} frames in dataset.".format(len(frames)))

    def __getitem__(self, idx):
        frame_time = self.frame_idxs[idx]
        points = self.frame_points[frame_time]   # N * 6
        labels = self.frame_labels[frame_time]   # N

        frame_pcd = op3.geometry.PointCloud(op3.utility.Vector3dVector(points[:,0:3]))
        center = frame_pcd.get_center()

        selected_point_idxs = np.random.choice(np.arange(points.shape[0]), self.num_points, replace=False)

        # normalize
        # selected_points = points[selected_point_idxs, :]  # num_points * 6
        # selected_labels = labels[selected_point_idxs]
        selected_points = points[selected_point_idxs, :]  # num_point * 6
        current_points = np.zeros((self.num_points, 6))  # num_point * 6
        current_points[:, 0] = selected_points[:, 0] - center[0]
        current_points[:, 1] = selected_points[:, 1] - center[1]
        current_points[:, 2] = selected_points[:, 2] - center[2]
        current_points[:, 0] = current_points[:, 0] / (self.frame_coord_max[frame_time][0] - self.frame_coord_min[frame_time][0])
        current_points[:, 1] = current_points[:, 1] / (self.frame_coord_max[frame_time][1] - self.frame_coord_min[frame_time][1])
        current_points[:, 2] = current_points[:, 2] / (self.frame_coord_max[frame_time][2] - self.frame_coord_min[frame_time][2])
        current_points[:, 3:6] = selected_points[:, 3:6]
        current_labels = labels[selected_point_idxs]
        
        #if self.transform is not None:
        #     selected_points, selected_labels = self.transform(selected_points, selected_labels)
        if self.transform is not None:
             current_points, current_labels = self.transform(current_points, current_labels)        
        return current_points, current_labels
        #return selected_points, selected_labels

    def __len__(self):
        return len(self.frame_idxs)

In [7]:
root = 'test/'
NUM_CLASSES = 46
NUM_POINT = 8192 #32768 #49152 #65536 #8192 #16384 #4096
BATCH_SIZE = 4

TRAIN_DATASET = KITTI360Dataset(data_root=root, num_points=NUM_POINT, transform=None)
trainDataLoader = torch.utils.data.DataLoader(TRAIN_DATASET, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)#, worker_init_fn = lambda x: np.random.seed(x+int(time.time())))

NameError: name 'KITTI360Dataset' is not defined

In [567]:
pts = next(iter(TRAIN_DATASET))[0]

In [568]:
pcd = open3d.geometry.PointCloud( open3d.utility.Vector3dVector(pts[:,0:3]) )

In [564]:
open3d.visualization.draw_geometries([pcd])

In [573]:
pcd2 = pcd.voxel_down_sample(0.01)
open3d.visualization.draw_geometries([pcd2])

In [570]:
pcd3 = pcd.voxel_down_sample(0.025)
open3d.visualization.draw_geometries([pcd3])

In [571]:
pcd4 = pcd3.voxel_down_sample(0.1)
open3d.visualization.draw_geometries([pcd4])

In [37]:
x = torch.tensor([[1,2,3],[-1,1,0]])
y = torch.tensor([2,5,1,5,1,2])
x.shape

torch.Size([2, 3])

In [38]:
x.sort(dim=-1)

torch.return_types.sort(
values=tensor([[ 1,  2,  3],
        [-1,  0,  1]]),
indices=tensor([[0, 1, 2],
        [0, 2, 1]]))

In [701]:
y[t]

array([2, 1])

In [85]:
x = [1,2,3]
y = [4,5,6]

In [12]:
x.extend(y)

AttributeError: 'Tensor' object has no attribute 'extend'

In [8]:
import torch
#x = torch.tensor([[1,2,3],[2,3,4],[5,6,7],[8,1,2]])
x = torch.tensor([[[1,2,3],[2,3,4],[5,6,7],[8,1,2]], [[1,2,3],[2,3,4],[5,6,7],[8,1,2]]])
print(x.shape)
x

torch.Size([2, 4, 3])


tensor([[[1, 2, 3],
         [2, 3, 4],
         [5, 6, 7],
         [8, 1, 2]],

        [[1, 2, 3],
         [2, 3, 4],
         [5, 6, 7],
         [8, 1, 2]]])

In [13]:
x = x.contiguous().view(-1, 3)
print(x.shape)
x

torch.Size([8, 3])


tensor([[1, 2, 3],
        [2, 3, 4],
        [5, 6, 7],
        [8, 1, 2],
        [1, 2, 3],
        [2, 3, 4],
        [5, 6, 7],
        [8, 1, 2]])

In [10]:
x.cpu().data.max(-1)[1].numpy()

array([2, 2, 2, 0, 2, 2, 2, 0])

In [11]:
x.cpu().data.max(-1)[1].numpy()

array([2, 2, 2, 0, 2, 2, 2, 0])

# Combine Scene

In [8]:
class opts():
    sequence = 0
    mode = 'semantic'
    max_bbox = 100

args = opts()

v = Kitti360Viewer3D(args.sequence, False)

pcdFile = v.annotation3DPly.pcdFileList[0]
pcds = v.loadWindow(pcdFile, args.mode)

# for i in range(1, 2):#44
#     pcdFile = v.annotation3DPly.pcdFileList[i]
#     pcds = pcds + v.loadWindow(pcdFile, args.mode)

# # Visualize the last sample scene
# open3d.visualization.draw_geometries([pcds])

Found 44 ply files in 2013_05_28_drive_0000_sync
Loading /Volumes/Erik's Files/Project/Datasets/KITTI-360/data_3d_semantics/2013_05_28_drive_0000_sync/static/000002_000385.ply 


In [10]:
open3d.visualization.draw_geometries([pcds])

In [15]:
p = pcds.voxel_down_sample(5)
open3d.visualization.draw_geometries([p])

In [ ]:
np.save('testing', np.asarray(pcds.points))
np.save('testingcols', np.asarray(pcds.colors))

In [18]:
import open3d
import numpy as np
import math
pts = np.load('testing.npy')
cols = np.load('testingcols.npy')
pcds = open3d.geometry.PointCloud( open3d.utility.Vector3dVector(pts))
pcds.colors = open3d.utility.Vector3dVector(cols)

# Get Camera View

In [20]:
cam2world = np.loadtxt('cam0_to_world.txt')

In [23]:
n=160
d = cam2world[n,1:]
#print(d)
#d = d.reshape(4,4)
print(d)
#d.shape

[ 9.124040e-01 -3.971888e-02  4.073610e-01  9.184893e+02 -4.092920e-01
 -8.683458e-02  9.082620e-01  3.710181e+03 -7.020000e-04 -9.954309e-01
 -9.548400e-02  1.160218e+02  0.000000e+00  0.000000e+00  0.000000e+00
  1.000000e+00]


In [4]:
# build visualization
vis = open3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcds)  # your point cloud goes here

vis = open3d.visualization.VisualizerWithKeyCallback()
vis.create_window()
view_ctl = vis.get_view_control()

vis.add_geometry(pcds)
cam = view_ctl.convert_to_pinhole_camera_parameters()
cam.extrinsic = np.linalg.inv(d) # where T is your matrix
view_ctl.convert_from_pinhole_camera_parameters(cam)
vis.run()
vis.destroy_window()

In [6]:
print(np.linalg.inv(d))

[[-4.17180158e-01 -9.08284229e-01 -3.12814598e-02  3.74213318e+03]
 [-8.25531786e-02  7.21491188e-02 -9.93971322e-01 -7.92802821e+01]
 [ 9.05066909e-01 -4.12083472e-01 -1.05081983e-01  7.40056610e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [13]:
box = open3d.geometry.AxisAlignedBoundingBox.create_from_points(pcds.points)

vol = open3d.geometry.OrientedBoundingBox.create_from_axis_aligned_bounding_box(box)
vol.extent = np.array([30,30,30])
vol.center = np.array([d[3],d[7],d[11]])
part = pcds.crop(vol)
open3d.visualization.draw_geometries([part])

In [24]:
box = open3d.geometry.AxisAlignedBoundingBox.create_from_points(pcds.points)

vol = open3d.geometry.OrientedBoundingBox.create_from_axis_aligned_bounding_box(box)
vol.extent = np.array([30,30,30])
vol.center = np.array([d[3],d[7],d[11]])

# angle = 14
# t = math.radians(angle)
# x = math.cos(t)
# y = math.sin(t)
# vol.rotate(np.array([[x,-y,0],[y,x,0],[0,0,1]]), vol.center)#pcd.get_center())
c = np.array(vol.center)
dd = d.reshape(4,4)
vol.rotate(dd[:3,:3], vol.center)

part = pcds.crop(vol)
#part.rotate(np.linalg.inv(dd)[:3,:3], vol.center)

r = np.linalg.inv(dd)
#part.rotate(-r[:3,:3], [0,0,0])
part.rotate(-r[:3,:3], r[:3,3])
open3d.visualization.draw_geometries([part])

In [15]:
#import natsort
lst = ['f_1','f_2','f_3','f_11','f_12']
lst2 = ['1_1','1_2','2_3','1_11','2_12']
print(lst)
#print(natsort.natsorted(lst))

['f_1', 'f_2', 'f_3', 'f_11', 'f_12']


In [9]:
lst = np.array(lst)

In [16]:
lst2.sort()
lst2

['1_1', '1_11', '1_2', '2_12', '2_3']

In [12]:
sorted(lst)

['f_1', 'f_11', 'f_12', 'f_2', 'f_3']